![https://pieriantraining.com/](../PTCenteredPurple.png)

In this exercise, you are going to create a database using the [Netflix Movies and TV Shows](https://www.kaggle.com/datasets/shivamb/netflix-shows?resource=download).

You can find the csv file in this directory (netflix_titles.csv)

In [1]:
with open("netflix_titles.csv", "r") as f:
    lines = f.readlines()

In [2]:
lines[:5]

['show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description\n',
 's1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."\n',
 's2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."\n',
 's3,TV Show,Ganglands,

**Task:** Create the table attributes and its schema. Use **show_id** as the partition key and **release_year** as range key.
Use *10* for both, read and write capacity units

In [3]:
table_name = "Shows"
attributes = [
    {
        "AttributeName": "show_id",
        "AttributeType" : "S"  # String
    },

    {
        "AttributeName": "release_year",
        "AttributeType" : "N"  # Number
    },

]

key_schema = [
    {
        'AttributeName': 'show_id',
        'KeyType': 'HASH'  # Hash Key for Primary Key
    },
    {
        'AttributeName': 'release_year',
        'KeyType': 'RANGE'  # Range key for sorting
    }
]

provisioned_throughput = {
    'ReadCapacityUnits': 10,
    'WriteCapacityUnits': 10
}


We want to use a [Global Secondary Index](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GSI.html#GSI.scenario), to be able to query for the Country ([Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb/client/create_table.html#:~:text=(string)%20%E2%80%93-,GlobalSecondaryIndexes,-(list)%20%E2%80%93))

**Task:** Create the Table with above schema and the secondary index


In [4]:
attributes.append({"AttributeName": "country", "AttributeType" : "S"})

In [5]:
attributes

[{'AttributeName': 'show_id', 'AttributeType': 'S'},
 {'AttributeName': 'release_year', 'AttributeType': 'N'},
 {'AttributeName': 'country', 'AttributeType': 'S'}]

In [1]:
import boto3
client = boto3.client("dynamodb", region_name="us-east-1")

In [7]:
response = client.create_table(
        TableName=table_name,
        AttributeDefinitions=attributes,
        KeySchema=key_schema,
        ProvisionedThroughput=provisioned_throughput,
        GlobalSecondaryIndexes=[
        {
            'IndexName': 'idx1',  # The name of the global secondary index. Must be unique only for this table.
            'KeySchema': [
               {
                  'AttributeName': 'country',
                  'KeyType': 'HASH'
               }
             ],
             'Projection': {
               'ProjectionType': 'ALL'  # Project all information
             },
             'ProvisionedThroughput': {
                  'ReadCapacityUnits': 10,
                  'WriteCapacityUnits': 10
             }
        }
    ],

)

**Task:** Read the data from the csv file to a list of dictionaries

In [8]:
import csv
data_list = []
with open("netflix_titles.csv", "r") as f:
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        data_list.append(row)

**Task:** Create the batch requests to upload the data. Note, that batch requests can only handle 25 items at once. Add the data types according to the elements.

In [12]:
items_to_upload = []

for item in data_list:
    put_request = {"PutRequest": {"Item": {}}}
    for key, value in item.items():
        
        if not value:  # Some fields are empty and dynamo db cannot handle empty fields for keys
            value = "None"

        try:
            float(value)
            put_request["PutRequest"]["Item"][key] = {'N': value}
        except ValueError:
            put_request["PutRequest"]["Item"][key] = {'S': value}
        
        
    items_to_upload.append(put_request)

    if len(items_to_upload) == 25:  # Upload the batch when it consists of 25 elements
        response = client.batch_write_item(RequestItems={
                                                table_name: items_to_upload  
                                            }
                                          )
        items_to_upload = []


As we set ProvisionedThroughput to a low value of 10 this might take a while as aws throttels the writing operations (but is very cheap)

**Task:** What is the name of the show with show id 10

In [13]:
response = client.query(TableName="Shows",
             KeyConditionExpression='show_id = :id',
            ExpressionAttributeValues={':id': {'S': "s10"}}
)
response["Items"][0]["title"]

{'S': 'The Starling'}

**Task:** What shows are from Germany?<br />
Hint: You need to pass IndexName to the query function

In [55]:
response = client.query(TableName="Shows",
             KeyConditionExpression='country = :id',
             IndexName='idx1',
            ExpressionAttributeValues={':id': {'S': "Germany"}}
)

In [20]:
[item["title"] for item in response["Items"]]

[{'S': 'Kidnapping Stella'},
 {'S': 'Hazel Brugger: Tropical'},
 {'S': 'Dieter Nuhr: Nuhr in Berlin'},
 {'S': 'Unorthodox'},
 {'S': 'Perfume'},
 {'S': 'Babylon Berlin'},
 {'S': 'The Houseboat'},
 {'S': 'A Perfect Crime'},
 {'S': 'Charité'},
 {'S': 'A Heavy Heart'},
 {'S': 'Frank Elstner: Just One Last Question'},
 {'S': 'How to Be Really Bad'},
 {'S': 'Biohackers'},
 {'S': 'Rock My Heart'},
 {'S': 'Berlin, I Love You'},
 {'S': 'Criminal: Germany'},
 {'S': 'How to Sell Drugs Online (Fast)'},
 {'S': 'Enissa Amani: Ehrenwort'},
 {'S': 'Holiday Secrets'},
 {'S': 'Nailed It! Germany'},
 {'S': 'Tribes of Europa'},
 {'S': '3 Türken & ein Baby'},
 {'S': 'Lommbock'},
 {'S': 'Dogs of Berlin'},
 {'S': 'Biking Borders'},
 {'S': 'Bibi & Tina'},
 {'S': "Freaks – You're One of Us"},
 {'S': 'The NSU-Complex'},
 {'S': 'We Are Young. We Are Strong.'},
 {'S': 'Berlin Calling'},
 {'S': 'Latte and the Magic Waterstone'},
 {'S': 'Berlin, Berlin: Lolle on the Run'},
 {'S': 'Bibi & Tina: Tohuwabohu Total'},
 

**Task:** What shows are from India and from 2021 or later?

In [53]:
items = []
response = client.scan(TableName="Shows",
            FilterExpression="release_year >= :num AND country = :c",  
            ExpressionAttributeValues={":num":{"N":"2021"}, ":c": {"S": "India"}}  
            )
items.extend(response["Items"])

## Code for paginating through results
while "LastEvaluatedKey" in response.keys():
    response = client.scan(TableName="Shows",
            FilterExpression="release_year >= :num AND country = :c",  
            ExpressionAttributeValues={":num":{"N":"2021"}, ":c": {"S": "India"}} ,
            ExclusiveStartKey=response["LastEvaluatedKey"])
    items.extend(response["Items"])


In [54]:
[item["title"] for item in items]

[{'S': 'Navarasa'},
 {'S': 'Bombay Begums'},
 {'S': 'Pagglait'},
 {'S': '99 Songs'},
 {'S': 'The Girl on the Train'},
 {'S': 'C/O Kaadhal'},
 {'S': 'Tuesdays & Fridays'},
 {'S': 'Kota Factory'},
 {'S': 'Ahaan'},
 {'S': 'The Disciple'},
 {'S': 'Aelay'},
 {'S': 'Sardar Ka Grandson'},
 {'S': 'Searching For Sheela'},
 {'S': 'Tribhanga - Tedhi Medhi Crazy'},
 {'S': 'The Big Day'},
 {'S': 'Chhota Bheem'},
 {'S': 'Ajeeb Daastaans'},
 {'S': 'Roohi'},
 {'S': 'Nayattu'},
 {'S': 'Mandela'},
 {'S': 'Irul'},
 {'S': 'Mighty Little Bheem: Kite Festival'},
 {'S': 'Thimmarusu'},
 {'S': 'Sarbath'},
 {'S': 'Red'},
 {'S': 'Pitta Kathalu'},
 {'S': 'Haseen Dillruba'},
 {'S': 'Lava Ka Dhaava'},
 {'S': 'Cinema Bandi'},
 {'S': 'Milestone'},
 {'S': 'Alma Matters'}]

**Task:** Can you also perform this operation using a query?

NO!
DynamoDB's Query operation on a Global Secondary Index doesn't support filtering based on a range condition directly in the KeyConditionExpression. Instead, you need to structure your data and queries differently to achieve the desired result.

In DynamoDB, global secondary indices are best suited for queries where you have a specific hash key value and you want to retrieve items with that hash key from the index. In the above case, querying for shows from a specific country and release year is only achievable using a scan

**Task:** Delete the table

In [57]:
client.delete_table(TableName="Shows")

{'TableDescription': {'TableName': 'Shows',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:472948420345:table/Shows',
  'TableId': '93375464-7815-482f-847b-c1edfa877660',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'MGD3H8K4F92BITS8CA9JMR6PPVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 17 Aug 2023 12:10:04 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '348',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MGD3H8K4F92BITS8CA9JMR6PPVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1336866181'},
  'RetryAttempts': 0}}